In [2]:
import pandas as pd

In [3]:
df = pd.read_pickle("results/final_chunks_complete_losses_WAE_LSTMDiscriminator_TCN_analog_feats_4_2_30_3_1.0_3_32_150_0.001_0.001_32.pkl")

In [7]:
df["train"].keys()

dict_keys(['reconstruction', 'critic'])

In [7]:
df.shape

(7116940, 21)

In [8]:
import os
import numpy as np
import torch as th
from torch import optim
from torch import nn
import torch.nn.functional as F
from torch.distributions.multivariate_normal import MultivariateNormal
import pickle as pkl
import tqdm
from ArgumentParser import parse_arguments
from models.LSTM_AAE import Encoder, Decoder, SimpleDiscriminator, LSTMDiscriminator, ConvDiscriminator
from torch.utils.data import Dataset, DataLoader
from models.LSTMAE import LSTM_AE
from models.LSTM_SAE import LSTM_SAE
from models.TCN_AE import TCN_AE
from models.TCN_AAE import Encoder_TCN, Decoder_TCN, SimpleDiscriminator_TCN, LSTMDiscriminator_TCN, ConvDiscriminator_TCN

In [9]:

encoders = dict(LSTM=Encoder,
                TCN=Encoder_TCN)

decoders = dict(LSTM=Decoder,
                TCN=Decoder_TCN)

In [14]:
device = th.device('cuda' if th.cuda.is_available() else 'cpu')
decoder = Encoder_TCN(4,16,0.2,2,hidden_dim=30, kernel_size=3).to(device)
encoder = Decoder_TCN(16,4,0.2,2,hidden_dim=30, kernel_size=3).to(device)
model = LSTMDiscriminator_TCN(4,0.2,n_layers=3,disc_hidden=32,kernel_size=3, window_size=1800).to(device)

In [15]:
decoder_path = "/home/ridaadam/Documents/metro-anomaly-ECML2023-main/predictive_maintance_metropt/results/final_chunks_offline_WAE_decoder_LSTMDiscriminator_TCN_analog_feats_4_2_30_3_1.0_3_32_150_0.001_0.001_32.pt"
encoder_path = "/home/ridaadam/Documents/metro-anomaly-ECML2023-main/predictive_maintance_metropt/results/final_chunks_offline_WAE_encoder_LSTMDiscriminator_TCN_analog_feats_4_2_30_3_1.0_3_32_150_0.001_0.001_32.pt"
discrimator_path = "/home/ridaadam/Documents/metro-anomaly-ECML2023-main/predictive_maintance_metropt/results/final_chunks_offline_WAE_discriminator_LSTMDiscriminator_TCN_analog_feats_4_2_30_3_1.0_3_32_150_0.001_0.001_32.pt"

In [16]:

# Load the weights into the models
decoder.load_state_dict(th.load(decoder_path, map_location=device))
encoder.load_state_dict(th.load(encoder_path, map_location=device))
model.load_state_dict(th.load(discrimator_path, map_location=device))

RuntimeError: Error(s) in loading state_dict for Encoder_TCN:
	size mismatch for output_layer.weight: copying a param with shape torch.Size([8, 30]) from checkpoint, the shape in current model is torch.Size([16, 30]).
	size mismatch for output_layer.bias: copying a param with shape torch.Size([8]) from checkpoint, the shape in current model is torch.Size([16]).

In [ ]:

# GLOBAL VARIABLES : ANOMALY THRESHOLD

def predict(sample, encoder, decoder, discriminator, TCN = True):
    # process sample to make it fit input size (probably th tensor)
    sample = th.tensor(sample.values, dtype=th.float32).to(device)
    latent_space = encoder(sample)
    if len(latent_space.shape) == 2:
        latent_space = latent_space.unsqueeze(1)
    critic_score = th.mean(discriminator(latent_space))
    if not TCN:
        latent_space = latent_space.repeat(1, sample.shape[1], 1).to(device)
    reconstruction = decoder(latent_space)
    reconstruction_error = F.mse_loss(reconstruction, sample)
    return reconstruction_error, critic_score








